In [1]:
# Run this first in a separate cell
import sys
print(f"Python version: {sys.version}")

Python version: 3.11.14 (main, Oct 21 2025, 18:27:30) [Clang 20.1.8 ]


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
import time
import glob
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("ADVANCED SELF-HEALING AI CYBER IMMUNE NETWORK")
print("=" * 80)
print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

ADVANCED SELF-HEALING AI CYBER IMMUNE NETWORK
TensorFlow version: 2.16.2
NumPy version: 1.26.4
Pandas version: 2.3.3


In [3]:
print("\nSTEP 2: Loading Dataset")
print("-" * 80)

df = pd.read_csv('../data/csvs/Obfuscated-MalMem2022.csv')

print(f"\nTotal dataset shape: {df.shape}")
print(f"Columns: {len(df.columns)}")
print("\nFirst few rows:")
print(df.head(3))


STEP 2: Loading Dataset
--------------------------------------------------------------------------------

Total dataset shape: (58596, 57)
Columns: 57

First few rows:
  Category  pslist.nproc  pslist.nppid  pslist.avg_threads  \
0   Benign            45            17           10.555556   
1   Benign            47            19           11.531915   
2   Benign            40            14           14.725000   

   pslist.nprocs64bit  pslist.avg_handlers  dlllist.ndlls  \
0                   0           202.844444           1694   
1                   0           242.234043           2074   
2                   0           288.225000           1932   

   dlllist.avg_dlls_per_proc  handles.nhandles  handles.avg_handles_per_proc  \
0                   38.50000              9129                    212.302326   
1                   44.12766             11385                    242.234043   
2                   48.30000             11529                    288.225000   

   ...  svcscan.

In [4]:
print("\nSTEP 3: Data Cleaning and Preprocessing")
print("-" * 80)

# Display basic info
print("\nDataset Info:")
print(f"Total rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"\nColumn names:")
print(df.columns.tolist())

# Check for missing values
print(f"\nMissing values per column:")
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "No missing values")

# Display data types
print(f"\nData types:")
print(df.dtypes.value_counts())

# Check if there's a label column
label_candidates = ['Label', 'Class', 'Category', 'Filename']
label_col = None
for col in label_candidates:
    if col in df.columns:
        label_col = col
        print(f"\nFound label column: '{label_col}'")
        break

if label_col is None:
    print("\nWARNING: No standard label column found. Using last column as label.")
    label_col = df.columns[-1]

# Display label distribution
print(f"\nLabel distribution in '{label_col}':")
print(df[label_col].value_counts())


STEP 3: Data Cleaning and Preprocessing
--------------------------------------------------------------------------------

Dataset Info:
Total rows: 58596
Total columns: 57

Column names:
['Category', 'pslist.nproc', 'pslist.nppid', 'pslist.avg_threads', 'pslist.nprocs64bit', 'pslist.avg_handlers', 'dlllist.ndlls', 'dlllist.avg_dlls_per_proc', 'handles.nhandles', 'handles.avg_handles_per_proc', 'handles.nport', 'handles.nfile', 'handles.nevent', 'handles.ndesktop', 'handles.nkey', 'handles.nthread', 'handles.ndirectory', 'handles.nsemaphore', 'handles.ntimer', 'handles.nsection', 'handles.nmutant', 'ldrmodules.not_in_load', 'ldrmodules.not_in_init', 'ldrmodules.not_in_mem', 'ldrmodules.not_in_load_avg', 'ldrmodules.not_in_init_avg', 'ldrmodules.not_in_mem_avg', 'malfind.ninjections', 'malfind.commitCharge', 'malfind.protection', 'malfind.uniqueInjections', 'psxview.not_in_pslist', 'psxview.not_in_eprocess_pool', 'psxview.not_in_ethread_pool', 'psxview.not_in_pspcid_list', 'psxview.not_

In [5]:
print("\nSTEP 4: Creating Binary Labels")
print("-" * 80)

# Create binary labels
if df[label_col].dtype == 'object':
    # If labels are strings (e.g., 'Benign', 'Malware', 'Spyware')
    def create_binary_label(x):
        x_str = str(x).lower()
        if 'benign' in x_str:
            return 0
        else:
            return 1
    
    df['Label'] = df[label_col].apply(create_binary_label)
else:
    # If labels are already numeric
    df['Label'] = df[label_col]

print("Binary label distribution:")
print(df['Label'].value_counts())
print(f"\nClass 0 (Benign): {(df['Label'] == 0).sum()} samples")
print(f"Class 1 (Malicious): {(df['Label'] == 1).sum()} samples")

# Check for class imbalance
class_ratio = df['Label'].value_counts()[1] / df['Label'].value_counts()[0]
print(f"Class imbalance ratio: {class_ratio:.2f}")

if df['Label'].value_counts().min() == 0:
    print("\nWARNING: One class has zero samples!")
    print("Please check your dataset and labeling logic.")


STEP 4: Creating Binary Labels
--------------------------------------------------------------------------------
Binary label distribution:
Label
0    29298
1    29298
Name: count, dtype: int64

Class 0 (Benign): 29298 samples
Class 1 (Malicious): 29298 samples
Class imbalance ratio: 1.00


In [6]:
print("\nSTEP 5: Feature Selection and Preparation")
print("-" * 80)

# Separate features and labels
# Drop non-numeric columns
numeric_df = df.select_dtypes(include=[np.number])

# Remove label from features if present
feature_cols = [col for col in numeric_df.columns if col not in ['Label', 'Unnamed: 0', 'index']]
X = numeric_df[feature_cols].values
y = df['Label'].values

print(f"Features shape: {X.shape}")
print(f"Labels shape: {y.shape}")

# Handle NaN and infinite values
print("\nHandling missing and infinite values...")
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)

# Check for constant features
print("\nChecking for constant features...")
constant_features = []
for i in range(X.shape[1]):
    if np.std(X[:, i]) == 0:
        constant_features.append(i)

if len(constant_features) > 0:
    print(f"Found {len(constant_features)} constant features, removing...")
    X = np.delete(X, constant_features, axis=1)

print(f"\nFinal feature matrix shape: {X.shape}")
print(f"Total samples: {len(y)}")



STEP 5: Feature Selection and Preparation
--------------------------------------------------------------------------------
Features shape: (58596, 55)
Labels shape: (58596,)

Handling missing and infinite values...

Checking for constant features...
Found 3 constant features, removing...

Final feature matrix shape: (58596, 52)
Total samples: 58596


In [7]:
print("\nSTEP 6: Train-Test Split")
print("-" * 80)

# Split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining label distribution:")
print(f"  Class 0: {(y_train == 0).sum()}")
print(f"  Class 1: {(y_train == 1).sum()}")
print(f"\nTest label distribution:")
print(f"  Class 0: {(y_test == 0).sum()}")
print(f"  Class 1: {(y_test == 1).sum()}")



STEP 6: Train-Test Split
--------------------------------------------------------------------------------
Training set: 46876 samples
Test set: 11720 samples

Training label distribution:
  Class 0: 23438
  Class 1: 23438

Test label distribution:
  Class 0: 5860
  Class 1: 5860


In [8]:
print("\nSTEP 7: Advanced Feature Engineering")
print("-" * 80)

class AdvancedFeatureEngineering:
    def __init__(self):
        self.scaler = RobustScaler()
        
    def engineer_features(self, X, fit=True):
        """Create advanced features from raw data"""
        if fit:
            X_scaled = self.scaler.fit_transform(X)
        else:
            X_scaled = self.scaler.transform(X)
        
        # Statistical features
        X_mean = np.mean(X_scaled, axis=1).reshape(-1, 1)
        X_std = np.std(X_scaled, axis=1).reshape(-1, 1)
        X_max = np.max(X_scaled, axis=1).reshape(-1, 1)
        X_min = np.min(X_scaled, axis=1).reshape(-1, 1)
        
        # Combine all features
        X_engineered = np.concatenate([X_scaled, X_mean, X_std, X_max, X_min], axis=1)
        return X_engineered

feature_engineer = AdvancedFeatureEngineering()
X_train_eng = feature_engineer.engineer_features(X_train, fit=True)
X_test_eng = feature_engineer.engineer_features(X_test, fit=False)

print(f"Original features: {X_train.shape[1]}")
print(f"Engineered features: {X_train_eng.shape[1]}")
print("Feature engineering complete")


STEP 7: Advanced Feature Engineering
--------------------------------------------------------------------------------
Original features: 52
Engineered features: 56
Feature engineering complete


In [9]:
print("\nSTEP 8: Building Advanced Neural Network")
print("-" * 80)

def build_advanced_model(input_dim):
    inputs = keras.Input(shape=(input_dim,))
    
    x = keras.layers.Dense(256, activation='relu')(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.4)(x)
    
    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.3)(x)
    
    x = keras.layers.Dense(64, activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    
    outputs = keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

advanced_model = build_advanced_model(X_train_eng.shape[1])
advanced_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.AUC(name='auc')]
)

print("Advanced neural network created")
print(f"Total parameters: {advanced_model.count_params():,}")


STEP 8: Building Advanced Neural Network
--------------------------------------------------------------------------------


2025-10-24 13:28:09.771648: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-10-24 13:28:09.771689: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-10-24 13:28:09.771703: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-10-24 13:28:09.771720: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-10-24 13:28:09.771733: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Advanced neural network created
Total parameters: 57,601


In [10]:
print("\nSTEP 9: Building Wasserstein GAN")
print("-" * 80)

class WGAN_GP:
    def __init__(self, data_dim, latent_dim=128):
        self.data_dim = data_dim
        self.latent_dim = latent_dim
        self.generator = self.build_generator()
        self.critic = self.build_critic()
        
    def build_generator(self):
        return keras.Sequential([
            keras.layers.Dense(256, activation='relu', input_dim=self.latent_dim),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(512, activation='relu'),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(1024, activation='relu'),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(self.data_dim, activation='tanh')
        ])
    
    def build_critic(self):
        return keras.Sequential([
            keras.layers.Dense(1024, activation='relu', input_dim=self.data_dim),
            keras.layers.Dropout(0.3),
            keras.layers.Dense(512, activation='relu'),
            keras.layers.Dropout(0.3),
            keras.layers.Dense(256, activation='relu'),
            keras.layers.Dense(1)
        ])
    
    def generate_samples(self, n_samples):
        noise = np.random.normal(0, 1, (n_samples, self.latent_dim))
        return self.generator.predict(noise, verbose=0)

wgan = WGAN_GP(X_train_eng.shape[1])
print("Wasserstein GAN created")



STEP 9: Building Wasserstein GAN
--------------------------------------------------------------------------------
Wasserstein GAN created


In [11]:
print("\nSTEP 10: Building Ensemble Detection System")
print("-" * 80)

class EnsembleDetector:
    def __init__(self):
        self.models = {
            'neural_net': None,
            'random_forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
            'gradient_boost': GradientBoostingClassifier(n_estimators=100, random_state=42)
        }
        self.weights = {'neural_net': 0.5, 'random_forest': 0.25, 'gradient_boost': 0.25}
        
    def fit(self, X, y, nn_model):
        print("Training ensemble components...")
        self.models['neural_net'] = nn_model
        print("  Training Random Forest...")
        self.models['random_forest'].fit(X, y)
        print("  Training Gradient Boosting...")
        self.models['gradient_boost'].fit(X, y)
        print("Ensemble training complete")
        
    def predict_proba(self, X):
        nn_pred = self.models['neural_net'].predict(X, verbose=0).flatten()
        rf_pred = self.models['random_forest'].predict_proba(X)[:, 1]
        gb_pred = self.models['gradient_boost'].predict_proba(X)[:, 1]
        
        ensemble_pred = (
            self.weights['neural_net'] * nn_pred +
            self.weights['random_forest'] * rf_pred +
            self.weights['gradient_boost'] * gb_pred
        )
        return ensemble_pred

ensemble = EnsembleDetector()
print("Ensemble system initialized")



STEP 10: Building Ensemble Detection System
--------------------------------------------------------------------------------
Ensemble system initialized


In [12]:
print("\nSTEP 11: Building Federated Learning System")
print("-" * 80)

class FederatedLearningSystem:
    def __init__(self, num_nodes=10, privacy_budget=1.0):
        self.num_nodes = num_nodes
        self.privacy_budget = privacy_budget
        self.global_model = None
        
    def split_data(self, X, y):
        node_data = []
        samples_per_node = len(X) // self.num_nodes
        for i in range(self.num_nodes):
            start = i * samples_per_node
            end = start + samples_per_node
            node_data.append((X[start:end], y[start:end]))
        return node_data
    
    def train_node(self, X_node, y_node, input_dim):
        model = build_advanced_model(input_dim)
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        model.fit(X_node, y_node, epochs=5, batch_size=64, verbose=0)
        return model
    
    def federated_averaging(self, models):
        avg_weights = []
        for weights_list in zip(*[m.get_weights() for m in models]):
            noise = np.random.laplace(0, self.privacy_budget, weights_list[0].shape)
            avg_weights.append(np.mean(weights_list, axis=0) + noise * 0.01)
        return avg_weights

fed_system = FederatedLearningSystem(num_nodes=10)
print(f"Federated system with {fed_system.num_nodes} nodes created")



STEP 11: Building Federated Learning System
--------------------------------------------------------------------------------
Federated system with 10 nodes created


In [13]:
print("\nSTEP 12: Building Reinforcement Learning Agent")
print("-" * 80)

class RLResponseAgent:
    def __init__(self, state_dim, action_dim=4):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.q_table = np.zeros((100, action_dim))
        self.epsilon = 0.1
        self.learning_rate = 0.1
        self.gamma = 0.95
        
    def choose_action(self, state_hash):
        if np.random.random() < self.epsilon:
            return np.random.randint(self.action_dim)
        return np.argmax(self.q_table[state_hash % 100])

rl_agent = RLResponseAgent(state_dim=X_train_eng.shape[1])
print("RL agent initialized")



STEP 12: Building Reinforcement Learning Agent
--------------------------------------------------------------------------------
RL agent initialized


In [14]:
print("\nSTEP 13: Building Self-Healing System")
print("-" * 80)

class AdvancedSelfHealingSystem:
    def __init__(self, model, ensemble, wgan, rl_agent):
        self.model = model
        self.ensemble = ensemble
        self.wgan = wgan
        self.rl_agent = rl_agent
        self.threat_history = deque(maxlen=1000)
        self.healing_log = []
        self.threat_threshold = 0.7
        
    def detect_threat(self, sample):
        ensemble_pred = self.ensemble.predict_proba(sample.reshape(1, -1))[0]
        
        if ensemble_pred > self.threat_threshold:
            threat_level = "CRITICAL" if ensemble_pred > 0.9 else "HIGH"
            return True, ensemble_pred, threat_level
        return False, ensemble_pred, "SAFE"
    
    def auto_heal(self, threat_data, threat_labels):
        print("  Initiating self-healing...")
        synthetic_samples = self.wgan.generate_samples(100)
        combined_X = np.vstack([threat_data, synthetic_samples])
        combined_y = np.hstack([threat_labels, np.ones(100)])
        self.model.fit(combined_X, combined_y, epochs=3, batch_size=64, verbose=0)
        self.healing_log.append({'timestamp': time.time(), 'status': 'HEALED'})
        print("  System healed")
    
    def adaptive_response(self, threat_level):
        actions = ['MONITOR', 'ISOLATE', 'BLOCK', 'HEAL']
        state_hash = hash(threat_level) % 100
        action_idx = self.rl_agent.choose_action(state_hash)
        return actions[action_idx]

print("Self-healing system initialized")



STEP 13: Building Self-Healing System
--------------------------------------------------------------------------------
Self-healing system initialized


In [15]:
print("\nSTEP 14: Training All Models")
print("-" * 80)

# Train neural network
print("Training deep neural network...")
history = advanced_model.fit(
    X_train_eng, y_train,
    validation_split=0.2,
    epochs=15,
    batch_size=128,
    verbose=1
)

# Train ensemble
print("\nTraining ensemble...")
ensemble.fit(X_train_eng, y_train, advanced_model)

# Federated learning
print("\nRunning federated learning...")
node_data = fed_system.split_data(X_train_eng, y_train)
node_models = []
for i, (X_n, y_n) in enumerate(node_data):
    print(f"  Node {i+1}/{fed_system.num_nodes}...")
    model = fed_system.train_node(X_n, y_n, X_train_eng.shape[1])
    node_models.append(model)

print("  Federated averaging...")
global_weights = fed_system.federated_averaging(node_models)
fed_system.global_model = build_advanced_model(X_train_eng.shape[1])
fed_system.global_model.set_weights(global_weights)
fed_system.global_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("All models trained")



STEP 14: Training All Models
--------------------------------------------------------------------------------
Training deep neural network...
Epoch 1/15


2025-10-24 13:28:10.608998: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


293/293 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.9269 - auc: 0.9772 - loss: 0.2471 - val_accuracy: 0.9899 - val_auc: 0.9941 - val_loss: 0.1222
Epoch 2/15
293/293 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.9886 - auc: 0.9946 - loss: 0.0703 - val_accuracy: 0.9929 - val_auc: 0.9970 - val_loss: 0.0847
Epoch 3/15
293/293 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.9914 - auc: 0.9975 - loss: 0.0449 - val_accuracy: 0.9946 - val_auc: 0.9980 - val_loss: 0.0577
Epoch 4/15
293/293 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.9923 - auc: 0.9975 - loss: 0.0401 - val_accuracy: 0.9959 - val_auc: 0.9985 - val_loss: 0.0479
Epoch 5/15
293/293 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9934 - auc: 0.9978 - loss: 0.0366 - val_accuracy: 0.9959 - val_auc: 0.9982 - val_loss: 0.0447
Epoch 6/15
293/293 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9939 - auc: 0.9978 - loss: 0.0347 - val_accuracy: 0.9963 - val_auc: 0.9986 - val_loss: 0.0403
Epoch 7/15
293/293 ━━━━━━━━━━━━━━━━━━━━ 5s 18

In [16]:
print("\nSTEP 15: Testing Self-Healing System")
print("-" * 80)

healing_system = AdvancedSelfHealingSystem(advanced_model, ensemble, wgan, rl_agent)

threats_detected = 0
for i in range(min(50, len(X_test_eng))):
    sample = X_test_eng[i]
    is_threat, confidence, level = healing_system.detect_threat(sample)
    
    if is_threat:
        threats_detected += 1
        action = healing_system.adaptive_response(level)
        if i < 3:
            print(f"  Threat {threats_detected}: {level} ({confidence:.2%}) -> {action}")

if threats_detected > 5:
    print(f"\n{threats_detected} threats detected, triggering self-heal...")
    healing_system.auto_heal(X_train_eng[:100], y_train[:100])

print(f"\nSimulation complete: {threats_detected} threats handled")



STEP 15: Testing Self-Healing System
--------------------------------------------------------------------------------
  Threat 1: CRITICAL (99.93%) -> MONITOR
  Threat 2: CRITICAL (100.00%) -> MONITOR

30 threats detected, triggering self-heal...
  Initiating self-healing...
  System healed

Simulation complete: 30 threats handled


In [17]:
print("\nSTEP 16: Final Evaluation")
print("-" * 80)

# Neural Network
loss, acc, auc = advanced_model.evaluate(X_test_eng, y_test, verbose=0)
print(f"\nDeep Neural Network:")
print(f"  Accuracy: {acc:.4f}")
print(f"  AUC: {auc:.4f}")

# Ensemble
ensemble_preds = ensemble.predict_proba(X_test_eng)
ensemble_acc = np.mean((ensemble_preds > 0.5) == y_test)
ensemble_auc = roc_auc_score(y_test, ensemble_preds)
print(f"\nEnsemble System:")
print(f"  Accuracy: {ensemble_acc:.4f}")
print(f"  AUC: {ensemble_auc:.4f}")

# Federated
fed_loss, fed_acc = fed_system.global_model.evaluate(X_test_eng, y_test, verbose=0)
print(f"\nFederated Model:")
print(f"  Accuracy: {fed_acc:.4f}")

# Classification report
y_pred = (ensemble_preds > 0.5).astype(int)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\n" + "=" * 80)
print("PROJECT COMPLETE")
print("=" * 80)
print(f"Neural Network Accuracy: {acc:.2%}")
print(f"Ensemble Accuracy: {ensemble_acc:.2%}")
print(f"Federated Accuracy: {fed_acc:.2%}")
print(f"Threats Detected: {threats_detected}")
print(f"Healing Operations: {len(healing_system.healing_log)}")
print("=" * 80)



STEP 16: Final Evaluation
--------------------------------------------------------------------------------

Deep Neural Network:
  Accuracy: 0.9868
  AUC: 0.9980

Ensemble System:
  Accuracy: 0.9978
  AUC: 1.0000

Federated Model:
  Accuracy: 0.5000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5860
           1       1.00      1.00      1.00      5860

    accuracy                           1.00     11720
   macro avg       1.00      1.00      1.00     11720
weighted avg       1.00      1.00      1.00     11720


PROJECT COMPLETE
Neural Network Accuracy: 98.68%
Ensemble Accuracy: 99.78%
Federated Accuracy: 50.00%
Threats Detected: 30
Healing Operations: 1


In [20]:
import pickle
import tensorflow as tf
import os

print("=" * 80)
print("SAVING TRAINED MODELS FOR DASHBOARD")
print("=" * 80)

# Save Advanced Neural Network (modern Keras format)
print("\nSaving advanced_model...")
try:
    advanced_model.save('advanced_model.keras')
    print("  ✅ Saved as advanced_model.keras")
except Exception as e:
    print(f"  ❌ Error: {e}")

# Save Ensemble System
print("\nSaving ensemble...")
try:
    ensemble_data = {
        'weights': ensemble.weights,
        'random_forest': ensemble.models['random_forest'],
        'gradient_boost': ensemble.models['gradient_boost']
    }
    with open('../models/ensemble.pkl', 'wb') as f:
        pickle.dump(ensemble_data, f)
    print("  ✅ Saved as ensemble.pkl")
except Exception as e:
    print(f"  ❌ Error: {e}")

# Save Federated Model (modern Keras format)
print("\nSaving federated_model...")
try:
    fed_system.global_model.save('federated_model.keras')
    print("  ✅ Saved as federated_model.keras")
except Exception as e:
    print(f"  ❌ Error: {e}")

# Save Feature Engineer (CORRECTED - save just the scaler)
print("\nSaving feature_engineer...")
try:
    # Save just the scaler component (which is pickleable)
    feature_engineer_data = {
        'scaler': feature_engineer.scaler,
        'type': 'RobustScaler'
    }
    with open('../models/feature_engineer.pkl', 'wb') as f:
        pickle.dump(feature_engineer_data, f)
    print("  ✅ Saved as feature_engineer.pkl")
except Exception as e:
    print(f"  ❌ Error: {e}")
    print(f"     Details: {str(e)}")

# Save model metadata
print("\nSaving model_metadata...")
try:
    metadata = {
        'accuracy': float(acc),
        'ensemble_accuracy': float(ensemble_acc),
        'federated_accuracy': float(fed_acc),
        'threat_threshold': 0.7,
        'num_features': int(X_train_eng.shape[1]),
        'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    with open('../models/model_metadata.pkl', 'wb') as f:
        pickle.dump(metadata, f)
    print("  ✅ Saved as model_metadata.pkl")
except Exception as e:
    print(f"  ❌ Error: {e}")

print("\n" + "=" * 80)
print("MODEL SAVING COMPLETE")
print("=" * 80)

# Verify files were created
print("\nVerifying saved files:")
files_to_check = [
    'advanced_model.keras',
    'ensemble.pkl',
    'federated_model.keras',
    'feature_engineer.pkl',
    'model_metadata.pkl'
]

for filename in files_to_check:
    if os.path.exists(filename):
        size = os.path.getsize(filename) / 1024  # KB
        print(f"  ✅ {filename} ({size:.2f} KB)")
    else:
        print(f"  ❌ {filename} - NOT FOUND")

print("\n" + "=" * 80)
print("You can now reload the dashboard!")
print("=" * 80)


SAVING TRAINED MODELS FOR DASHBOARD

Saving advanced_model...
  ✅ Saved as advanced_model.keras

Saving ensemble...
  ✅ Saved as ensemble.pkl

Saving federated_model...
  ✅ Saved as federated_model.keras

Saving feature_engineer...
  ✅ Saved as feature_engineer.pkl

Saving model_metadata...
  ✅ Saved as model_metadata.pkl

MODEL SAVING COMPLETE

Verifying saved files:
  ✅ advanced_model.keras (722.63 KB)
  ✅ ensemble.pkl (639.95 KB)
  ✅ federated_model.keras (269.85 KB)
  ✅ feature_engineer.pkl (1.22 KB)
  ✅ model_metadata.pkl (0.17 KB)

You can now reload the dashboard!


In [21]:
import pickle

# Load metadata
with open('../models/model_metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

print("=" * 80)
print("ACTUAL PROJECT METRICS")
print("=" * 80)
print(f"Neural Network Accuracy: {metadata['accuracy']:.4f}")
print(f"Ensemble Accuracy: {metadata['ensemble_accuracy']:.4f}")
print(f"Federated Learning Accuracy: {metadata['federated_accuracy']:.4f}")
print(f"Number of Features: {metadata['num_features']}")
print(f"Training Date: {metadata['training_date']}")
print("=" * 80)

ACTUAL PROJECT METRICS
Neural Network Accuracy: 0.9868
Ensemble Accuracy: 0.9978
Federated Learning Accuracy: 0.5000
Number of Features: 56
Training Date: 2025-10-24 14:13:36
